In [26]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mexwell/famous-birthdays")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\BineshMahato\.cache\kagglehub\datasets\mexwell\famous-birthdays\versions\2


In [28]:
import ipywidgets as widgets
from PIL import Image
import io

# Upload image widget (hidden initially)
upload = widgets.FileUpload(
    accept='image/*',  # Accept only images
    multiple=False  # Allow single file upload
)

# Display widget
display(upload)

# Function to process the uploaded image
def on_upload_change(change):
    # Ensure a file was uploaded
    if upload.value:
        # Get the uploaded image (the file is in a dictionary)
        file_info = upload.value[0]
        image_data = file_info['content']
        
        # Convert bytes to a PIL Image object
        image = Image.open(io.BytesIO(image_data))
        
        # Optionally: Display the image
        image.show()
        
        # You can now pass the image to a model for emotion detection, age estimation, etc.
        # For example:
        # emotion = detect_emotion(image)
        # age = estimate_age(image)
        # You could use the emotion or age to adjust your message generation logic

        # For demonstration, let's print the file name
        print(f"Uploaded image: {file_info['metadata']['name']}")
        
        # You could also pass this image to a model for further analysis (e.g., emotion recognition, age prediction)

# Set the callback function to run when an image is uploaded
upload.observe(on_upload_change, names='value')


FileUpload(value=(), accept='image/*', description='Upload')

In [51]:
import os
import ipywidgets as widgets
from IPython.display import display
import time

# Directory to save uploaded files
upload_directory = 'uploaded_images'

# Create directory if it doesn't exist
if not os.path.exists(upload_directory):
    os.makedirs(upload_directory)

# Upload image widget (hidden initially)
upload = widgets.FileUpload(
    accept='image/*',  # Accept only images
    multiple=False  # Allow single file upload
)

# Upload button to trigger file selection
upload_button = widgets.Button(description="Select Image to Upload")

# Loading bar widget
loading_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Uploading:',
    bar_style='info',  # can be 'success', 'info', 'warning', 'danger'
    style={'description_width': 'initial'},
    orientation='horizontal'
)

# Function to show the file upload widget when the button is clicked
def on_upload_button_click(b):
    # Show the file upload widget
    upload.layout.visibility = 'visible'  
    # Hide the upload button after it's clicked
    upload_button.layout.visibility = 'hidden'  

# Bind the button click event
upload_button.on_click(on_upload_button_click)

# Function to save and display the uploaded file details
def save_uploaded_file(change):
    # Check if a file is uploaded
    uploaded_file = upload.value
    if uploaded_file:
        # Debugging: Print the entire structure of the uploaded file
        print("Debug: Uploaded File Structure:", uploaded_file)

        # The `upload.value` can be a list or tuple in some cases, so we handle both cases
        if isinstance(uploaded_file, tuple):
            uploaded_file = uploaded_file[0]  # Get the first item (since there's only one file)

        # Now uploaded_file is a dictionary with keys as filenames
        for filename, file_info in uploaded_file.items():
            # Debugging: Check the type of file_info
            print(f"Debug: File info type: {type(file_info)}")
            print(f"Debug: File info content: {file_info}")
            
            # Handle unexpected types
            if isinstance(file_info, dict):
                file_content = file_info['content']  # Extract file content
            elif isinstance(file_info, str):
                # If it's a string, treat it as the content directly
                file_content = file_info.encode()  # Convert the string content to bytes
            else:
                raise ValueError(f"Unexpected file_info format: {type(file_info)}")

            # Ensure the saved file has a .jpg extension, regardless of the uploaded file's extension
            file_path = os.path.join(upload_directory, f"{os.path.splitext(filename)[0]}.jpg")
            
            # Show progress bar during saving
            loading_bar.value = 0  # Reset progress bar to 0 before uploading
            
            # Simulate the upload process (you can remove this simulation in production)
            for i in range(1, 101):
                time.sleep(0.05)  # Simulate some delay during the upload
                loading_bar.value = i  # Update the progress bar
            
            # Save the file content to the specified file path
            with open(file_path, 'wb') as f:
                f.write(file_content)  # Write the file content as bytes
            
            # Update progress bar to 100% when finished
            loading_bar.value = 100  # Set it to 100% when upload is complete
            print(f"File saved successfully at: {file_path}")
            print(f"File name: {filename}")
            
            # Display the file content (for verification purposes)
            print(f"File Content (First 100 bytes): {file_content[:100]}")  # Display first 100 bytes of content
            print(f"File size: {len(file_content)} bytes")  # Display file size
            
            # Hide the progress bar after a brief moment
            time.sleep(1)
            loading_bar.layout.visibility = 'hidden'  # Hide the progress bar after completion

# Bind the save function to the widget's value change event
upload.observe(save_uploaded_file, names='value')

# Display the widgets
display(upload_button)
display(upload)
display(loading_bar)

# Initially, the file upload widget is hidden, and only the "Select Image to Upload" button is visible
upload.layout.visibility = 'hidden'  # Hide the file upload widget initially


Button(description='Select Image to Upload', style=ButtonStyle())

FileUpload(value=(), accept='image/*', description='Upload')

IntProgress(value=0, bar_style='info', description='Uploading:', style=ProgressStyle(description_width='initia…

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer  # For imputing missing values

# Load the dataset from the CSV file
data_path = r"C:\Users\BineshMahato\.cache\kagglehub\datasets\mexwell\famous-birthdays\versions\2\birthdays.csv"
df = pd.read_csv(data_path)

# Display the first few rows of the dataset to check its structure
print(df.head())

# Handle missing values (impute numeric columns)
imputer = SimpleImputer(strategy='most_frequent')  # Use most frequent value for categorical data
df[['firstname', 'birthDate', 'birthMonth', 'birthDay', 'zodiac']] = imputer.fit_transform(df[['firstname', 'birthDate', 'birthMonth', 'birthDay', 'zodiac']])

# Verify missing values are handled
print(df.isnull().sum())

# Encode the 'zodiac' column as categorical data
df['zodiac'] = pd.Categorical(df['zodiac'])
df['zodiac_code'] = df['zodiac'].cat.codes

# Convert 'birthMonth' to numeric value
df['birthMonth'] = pd.to_datetime(df['birthMonth'], errors='coerce').dt.month

# Drop any rows with missing values in 'birthMonth'
df = df.dropna(subset=['birthMonth'])

# Now, let's use 'birthMonth' as a feature for prediction
X = df[['birthMonth']]  # Feature: birthMonth
y = df['zodiac_code']  # Target: zodiac_code

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Example input: Let's predict the zodiac sign based on the birth month '5' (May)
input_data = pd.DataFrame([[5]], columns=['birthMonth'])

# Predict the zodiac (output will be in numeric code)
predicted_zodiac_code = model.predict(input_data)

# Convert numeric prediction back to original zodiac category
inverse_prediction = df['zodiac'].cat.categories[predicted_zodiac_code][0]
print(f'Predicted zodiac sign: {inverse_prediction}')


  lastname firstname  articleNum   birthDate  birthMonth  birthDay     zodiac
0  Aaliyah       NaN           0  1979-01-16         1.0      16.0  Capricorn
1    Aaron      Hank          46  1934-02-05         2.0       5.0   Aquarius
2   Abacha      Sani           2  1943-09-20         9.0      20.0      Virgo
3   Abbado   Claudio           9  1933-06-26         6.0      26.0     Cancer
4    Abbas   Mahmoud         306  1935-03-26         3.0      26.0      Aries
lastname      0
firstname     0
articleNum    0
birthDate     0
birthMonth    0
birthDay      0
zodiac        0
dtype: int64
Predicted zodiac sign: Capricorn


In [39]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Encode input text
input_text = "Happy birthday to My friend."
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text with adjusted parameters
output = model.generate(
    inputs["input_ids"], 
    max_length=100, 
    num_return_sequences=1, 
    no_repeat_ngram_size=2,  # Prevent repeating 2-grams
    temperature=0.7,  # Control randomness
    top_p=0.9,  # Nucleus sampling
    top_k=50,  # Top-k sampling, limit to the top 50 tokens
    do_sample=True  # Use sampling instead of greedy generation
)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy birthday to My friend.

You might have noticed that my first email to you is a question that I asked myself, and it's an obvious one:
. . .
…
"What's the point in having to deal with this kind of thing?"
: . I'm just not a fan of this. And I don't like it at all. So let me tell you something: I've always been a big fan. I think it is the best way to have


In [40]:
from transformers import pipeline
import pandas as pd
import json

# Since the model and tokenizer have already been loaded:
# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# Create the text-generation pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Load the dataset from the CSV file (you can use it later for more personalization)
data_path = r"C:\Users\BineshMahato\.cache\kagglehub\datasets\mexwell\famous-birthdays\versions\2\birthdays.csv"
df = pd.read_csv(data_path)

# Assuming you know the user's birth month and want to predict the zodiac sign
user_birth_month = 1  # Example: May

# Zodiac map based on the birth month
zodiac_map = {
    1: 'Capricorn', 2: 'Aquarius', 3: 'Pisces', 4: 'Aries', 5: 'Taurus',
    6: 'Gemini', 7: 'Cancer', 8: 'Leo', 9: 'Virgo', 10: 'Libra',
    11: 'Scorpio', 12: 'Sagittarius'
}

# Get the zodiac sign based on birth month
user_zodiac_sign = zodiac_map[user_birth_month]

# Create the input prompt using the zodiac sign
user_input = f"Happy Birthday, {user_zodiac_sign}! Wishing you success in your career and happiness in your life."

# Generate a personalized birthday message using GPT-2
generated_message = generator(user_input, max_length=100, num_return_sequences=1)

# Print the generated birthday message
print(generated_message[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Happy Birthday, Capricorn! Wishing you success in your career and happiness in your life. You love your job. And in your heart, your heart! In your spirit, Capricorn! You will protect our country so we can live peacefully together and honor your family. I love you, Capricorn. I love you, Capricorn, very much.

Your gift will be given to me. Now, Capricorn! I love you, Capricorn.


In [58]:
from transformers import pipeline
import json

# Load GPT-2 model
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate personalized birthday message
def generate_birthday_message(name):
    prompt = f"Write a birthday message for {name}:"
    message = generator(prompt, max_length=50, num_return_sequences=1)
    return message[0]['generated_text']

# Function to generate future prediction
def generate_prediction(name):
    prompt = f"Write a prediction for {name} in 10 years:"
    prediction = generator(prompt, max_length=150, num_return_sequences=1)
    return prediction[0]['generated_text']

# Photo path (you can replace this with the actual path of the uploaded image)
photo_path = "memory_2.jpeg"  # Replace with actual path

# Generate personalized content
name = "Binesh"  
birthday_message = generate_birthday_message(name)
prediction = generate_prediction(name)


# Collecting data into a dictionary
capsule_data = {
    'memories': [photo_path],  # Add the photo path
    'predictions': prediction,
    'birthday_message': birthday_message,
}

# Saving the time capsule data as a JSON file
with open('time_capsule.json', 'w') as json_file:
    json.dump(capsule_data, json_file)

print("Time capsule saved successfully with the photo and content.")

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Time capsule saved successfully with the photo and content.


In [61]:
from PIL import Image
import json

# Load the time capsule data from the JSON file
with open('time_capsule.json', 'r') as json_file:
    capsule_data = json.load(json_file)

# Extract the data
photo_path = capsule_data['memories'][0]
birthday_message = capsule_data['birthday_message']
prediction = capsule_data['predictions']

# Display the image using PIL
img = Image.open(photo_path)
img.show()  # This will open the photo in the default viewer

# Print the generated content
print(f"Birthday Message: {birthday_message}")
print(f"Prediction: {prediction}")


Birthday Message: Write a birthday message for Binesh:

<b>Binesh</b>

Then

<string> "</string>

When you press <c,c>on</c>, select any letter and
Prediction: Write a prediction for Binesh in 10 years:

if (i.e. (I.e. we'll have a 10 year chance, or if he can actually do that to the people who make things in his life like not getting raped? We know he will, but why would he want to do that now when they haven't raped him? Why do we care if he's going to survive and his life in general would really be a mess for the next decade if that all changed?)

There are ways you can take one idea from the present and rewrite it to avoid a future that seems more like 100% predictable. Let's make a prediction about whether P-K can survive the B2B and if it


In [21]:
from PIL import Image, ImageDraw, ImageFont


# Load memory images
memory_images = ['memory_1.jpg', 'memory_2.jpeg']  # Replace with actual file paths
images = [Image.open(img_path) for img_path in memory_images]

# Create a blank white image for the canvas
canvas_width = 800
canvas_height = 600
canvas = Image.new('RGB', (canvas_width, canvas_height), color='white')
draw = ImageDraw.Draw(canvas)

# Resize images to fit within the canvas
image_width = int(canvas_width / 2)
image_height = int(canvas_height / 2)
resized_images = [img.resize((image_width, image_height)) for img in images]

# Paste images onto the canvas (side by side)
x_offset = 0
for img in resized_images:
    canvas.paste(img, (x_offset, 0))
    x_offset += image_width

# Add the generated prediction and birthday message text
font = ImageFont.load_default()  # You can use a custom font file if needed
text_margin = 10

# Add the prediction text
draw.text((text_margin, canvas_height - 100), prediction, fill='black', font=font)
# Add the birthday message text
draw.text((text_margin, canvas_height - 40), birthday_message, fill='black', font=font)

# Save or display the time capsule image
canvas.save('time_capsule_image.jpg')  # Save the image
canvas.show()  # Display the image


In [ ]:
import matplotlib.pyplot as plt

# Example data: Career progression prediction over years
years = [2024, 2025, 2026, 2027]
predictions = ['Engineer', 'Manager', 'Lead Engineer', 'Director']

plt.plot(years, predictions, marker='o')
plt.xlabel('Year')
plt.ylabel('Career Progression')
plt.title('Career Progression Over Time')
plt.show()
